In [ ]:
# magic
%matplotlib inline

# imports
import random as rand
from math import *

import matplotlib
import matplotlib.pyplot as plt
# import numpy as np
import networkx as nx

from shapely.geometry import *
from shapely.affinity import *
from shapely.prepared import prep

# my libraries
from mcr import *

# make figures large enough
from pylab import rcParams
rcParams['figure.figsize'] = (10, 10)

# MCR creation and basics

In [ ]:
# Create an empty square
square = MCR()

# Add a couple of shapes to the square
shape1 = Polygon([(0.2,0.2), (0.2, 0.7), (0.7,0.7), (0.7,0.2)])
shape2 = Polygon([(0.5,0.15), (1.0,0.15), (.75, .87)])
shape3 = Polygon([(0.0, 0.2), (0.0, 0.5), (0.3, 0.5), (0.3,0.2)])

square.add_obstacle(shape1)
square.add_obstacle(shape2)
square.add_obstacle(shape3)

# change its color?! Why not?
shape1.facecolor = 'pink'

# Add a start and goal
square.start = Point(0.05, 0.05)
square.goal  = Point(0.95, 0.95)

In [ ]:
# Display the MCR
square.plot_obstacles(labels=True)

In [ ]:
square.show(labels=True)

## Loading from an SVG

In [ ]:
# This should fail with the proper error
try:
    fail = MCR(svg='missing.svg')
except ValueError:
    pass

In [ ]:
shapes = MCR(svg='shapes.svg')

In [ ]:
shapes.create_graph()

In [ ]:
shapes.show()

In [ ]:
# The above is equivalent to:
shapes.plot_obstacles()
shapes.plot_graph(labels=False)
shapes.setup_axes()
plt.show()

In [ ]:
straws = MCR(svg='straws.svg')

In [ ]:
straws.plot_obstacles()

In [ ]:
straws.plot_overlapped_obstacles()

In [ ]:
straws.create_graph()

In [ ]:
straws.show()

In [ ]:
# how about circles from svg's?
laby4 = MCR(svg='labyrinth 4.svg')
laby4.show()

In [ ]:
# There are still some bugs. Notice that the unoverlapped sections in 
# the middle shape should connect to the whitespaces above and below,
# but only connect to one or the other.
# Notice too that the leftmost and second-to-rightmost overlaps have
# superthin shapes on their top borders.
laby3 = MCR(svg='labyrinth 3.svg')
laby3.show()

# Other functions in `mcr.py`

In [ ]:
f = random_MCR(30)
f.plot_obstacles(labels=True)
f.show()

In [ ]:
### highlight overlapping obstacles
triples = [o_o for o_o in f.overlapped_obstacles if len(o_o.cover) >= 3]

# find the overlappings and mark them
for o_o in triples:
    if len(o_o.cover) > 4:
        o_o.facecolor = 'red'
    elif len(o_o.cover) > 3:
        o_o.facecolor = 'orange'
    elif len(o_o.cover) > 2:
        o_o.facecolor = 'yellow'


# first draw everything
f.plot_obstacles()

MCR.plot_shapes(triples)

# show the graph as well
f.plot_graph(labels=False)
f.setup_axes()
plt.show()

In [ ]:
### highlight obstacles in the direct path from start to goal

# first draw everything
f.plot_obstacles()

# created the LineString and draw it
path = LineString([Point(f.start), Point(f.goal)])
MCR.plot_linestrings([path])

# find the crossed obstacles and show them
crossed = [o_o for o_o in f.overlapped_obstacles if path.crosses(o_o)]
for o_o in crossed:
    o_o.facecolor = 'yellow'
MCR.plot_shapes(crossed)

# show the graph as well
f.plot_graph(labels=False)
f.setup_axes()
plt.show()

# Subgraphs

In [ ]:
basic = MCR(svg='basic.svg')
basic.show(labels=True)

In [ ]:
# We determine the minimum cover is {1, 2} some other way
path = basic.shortest_covered_path(basic.start.wkt, basic.goal.wkt, cover=set([1,2]))

basic.plot_obstacles()
basic.plot_graph()
basic.plot_subgraph(path)
plt.show()

In [ ]:
# We determine the minimum cover is {1, 2} some other way
paths = basic.shortest_covered_paths(basic.start.wkt, basic.goal.wkt, cover=set([1,2,3]))

basic.plot_obstacles()
basic.plot_graph()
basic.plot_subgraph(paths)
plt.show()

In [ ]:
def solve_brute_force(mcr, max_steps=1E5):
    """
    Tries every combination of obstacles to find the minimum path.
    """
    
    # prepare the graph
    mcr.create_graph()
    start = mcr.start.wkt
    goal = mcr.goal.wkt
    
    for i in range(mcr.size + 1):
        # create all powersets of size i
        print('Trying all {} combinations of size {}'.format(2**i, i))
        P_i = itertools.combinations(range(1, mcr.size + 1), i) # nodes start at '1'
        
        for C in P_i:
            G = nx.Graph(data=mcr.graph)  # a fresh copy
            cover = set(C)
            # remove the cover from G's obstacles
            for n in dict(G.node): # use a copy!
                if G.node[n]['cover'] - cover != set():
                    G.remove_node(n)
                    
            # is there a path?
            if nx.has_path(G, start, goal):
               return cover

In [ ]:
f = random_MCR(25)
cover = solve_brute_force(f)
print(cover)

path = f.shortest_covered_path(f.start.wkt, f.goal.wkt, cover=cover)

f.plot_obstacles()
f.plot_graph()
f.plot_subgraph(path)
MCR.setup_axes()
plt.show()

In [ ]:
f.show()